# Estudio de los datos de Ventas de una Ferretería

Autor: Diana Chacón Ocariz

## Contexto:

Se trata de una pequeña ferretería que maneja un poco más de 3.000 productos distintos. Poseen un software de gestión genérico que les provee una gran cantidad de reportes, básicamente tablas con números, díficiles de analizar (un reporte puede constar de varias decenas de páginas).


## Objetivos del negocio:

**Tener más visibilidad sobre las ventas para poder mejorar el proceso de compras y la toma de decisiones en general:** 

    - Poder analizar objetivamente las ventas
    - Determinar los productos que podrían entrar en rotura de stock al final de un período
    - Identificar los productos menos vendidos
    - Identificar patrones en el comportamiento de las ventas
    

## Objetivos académicos:

    - Estudiar un caso real, con datos reales y cuyo resultado pueda ayudar a alguien a resolver un problema. 
    - Demostrar que la ciencia de datos también puede ayudar a las PYMES
    - Conocer y practicar el uso de herramientas de ciencia de datos
    
## Fuentes de datos:

Los datos provienen de reportes sacados del software de gestión de la empresa. Se trata de archivos .xls que contienen sólo los datos de reportes sobre ventas por producto (2021 y 2022). 

In [1]:
# Librerías utilizadas

import os
import glob
from pathlib import Path

import datetime 

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import altair as alt

%matplotlib inline

## Notebook 1: Carga y Limpieza de Datos:

Una vez leídos los datos y luego de una primera limpieza de los DF, se guardarán en archivos **.parquet** que serán utilizados más tarde en el EDA.

También se guardarán en archivos **.xlsx** que se utilizarán en la creación de reportes en Google Data Studio para failicitar el análisis por parte de los gerentes de la ferretería.

In [2]:
BASE_DIR = Path.cwd()
BASE_DIR

PosixPath('/home/diana/Documentos/Ciencia de Datos/Proyecto Ventas')

In [3]:
# Lectura de los archivos y creación de un DF con todos los datos
    
def read_files(FILES, TYPE_FILE=True):

    df = pd.DataFrame()

    for filename in glob.glob(f"{BASE_DIR / FILES}"):
        df_aux = pd.read_excel(filename)
        
        if TYPE_FILE:
            type_file = Path(filename).name[4:6]
            df_aux['Tipo'] = type_file

        df = pd.concat([df, df_aux])

    return df

## Carga de Datos de Ventas

In [4]:
FILES_VENTAS = 'datos/in/art*.xls'
FILES_VENTAS

'datos/in/art*.xls'

In [5]:
[Path(filename).name for filename in glob.glob(f"{BASE_DIR / FILES_VENTAS}")]

['art_fa_2022.xls', 'art_ne_2021.xls', 'art_ne_2022.xls', 'art_fa_2021.xls']

In [6]:
%%time

df = read_files(FILES_VENTAS)

df

CPU times: user 1.56 s, sys: 24.1 ms, total: 1.59 s
Wall time: 1.59 s


,Número,Reng,Emisión,Cliente,Vendedor,Almacén,Cantidad,Unid.,Precio Unitario,Monto Base,I.V.A.,Otros,Neto,Tipo
0,00001,NaN,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
1,0000006366,1,2022-01-03 09:00:00,304459.0,1.00,1.00,1.0,UNI,8.64,8.64,1.38,0.0,10.02,fa
2,0000006388,1,2022-01-07 11:40:00,18419125.0,6.00,1.00,1.0,UNI,8.91,8.91,1.43,0.0,10.34,fa
3,Sub-Totales:,2,17.55,2.81,0.00,20.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
4,00005,NaN,"NIPLE PLASTICO 1"" * 13CMS",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17659,0000004458,15,2021-05-15 11:10:00,15862673,7.00,1.00,1.0,UNI,10.00,10.00,1.60,0.0,11.60,fa
17660,0000005244,2,2021-06-15 10:32:00,15927422,1.00,1.00,1.0,UNI,4.55,4.55,0.00,0.0,4.55,fa
17661,0000005711,3,2021-07-14 10:52:00,13306742,14.00,1.00,1.0,UNI,330.00,330.00,52.80,0.0,382.80,fa
17662,Sub-Totales:,4,366.47,57.91,0.00,424.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa


In [7]:
df.describe()

,Vendedor,Almacén,Cantidad,Precio Unitario,Monto Base,I.V.A.,Otros,Neto
count,33640.000000,33640.000000,28925.000000,28925.000000,28925.000000,28925.000000,28925.000000,28925.000000
mean,8.006907,81.789050,4.566491,19.517480,26.960769,0.692567,0.000006,26.880727
std,4.591927,4574.323653,21.901486,97.215986,137.228900,7.324589,0.000764,113.978329
min,0.000000,0.010000,0.020000,0.010000,0.010000,0.000000,0.000000,0.010000
25%,7.000000,1.000000,1.000000,1.980000,2.600000,0.000000,0.000000,2.690000
50%,10.000000,1.000000,1.000000,5.555550,7.500000,0.000000,0.000000,7.650000
75%,11.000000,1.000000,3.000000,14.588750,20.500000,0.000000,0.000000,21.000000
max,14.000000,618970.850000,2000.000000,5287.500000,11340.010000,846.000000,0.120000,6133.500000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38351 entries, 0 to 17663
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Número            38351 non-null  object 
 1   Reng              34207 non-null  object 
 2   Emisión           38351 non-null  object 
 3   Cliente           33640 non-null  object 
 4   Vendedor          33640 non-null  float64
 5   Almacén           33640 non-null  float64
 6   Cantidad          28925 non-null  float64
 7   Unid.             28925 non-null  object 
 8   Precio Unitario   28925 non-null  float64
 9   Monto Base        28925 non-null  float64
 10  I.V.A.            28925 non-null  float64
 11  Otros             28925 non-null  float64
 12  Neto              28925 non-null  float64
 13  Tipo              38351 non-null  object 
dtypes: float64(8), object(6)
memory usage: 4.4+ MB


## Limpieza y transformación de los datos:

Aunque los datos no tienen ningún formato especial de Excel, tienen la forma de un reporte con totales, subtotales y datos agrupados por producto

- **Eliminación de totales:** Eliminamos las lineas que continen "total" ya que son los totales y subtotales de los reportes
- **Eliminación de información no relevante:** Conservaremos solo las siguientes columnas: Número, Emisión, Cliente, Vendedor, Cantidad, Neto y Tipo 
- **Construcción del DF definitvo:** Recorreremos el DF para recuperar la información por producto y crearemos un nuevo DF con los datos definitivos
- **Cambio tipos columnas:** Cambiamos el tipo a la columna Vendedor para que sea de tipo entero. Transformamos el campo de fecha para que sea de tipo datetime. Transformamos las columnas num y cliente a str

### Eliminación de totales y subtotales

In [9]:
# Buscamos las filas de totales y subtotales para eliminarlas
df[df.Número.str.contains('Totales')] 

,Número,Reng,Emisión,Cliente,Vendedor,Almacén,Cantidad,Unid.,Precio Unitario,Monto Base,I.V.A.,Otros,Neto,Tipo
3,Sub-Totales:,2,17.55,2.81,0.00,20.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
7,Sub-Totales:,25,133.08,21.29,0.00,154.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
11,Sub-Totales:,2,53.11,0.0,0.00,53.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
14,Sub-Totales:,2,17.74,0.0,0.00,17.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
19,Sub-Totales:,3,9.64,1.23,0.00,10.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17630,Sub-Totales:,1,28.5,4.56,0.00,33.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
17633,Sub-Totales:,1,6.25,0,0.00,6.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
17656,Sub-Totales:,86.68,637.51,0,0.00,637.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
17662,Sub-Totales:,4,366.47,57.91,0.00,424.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa


In [10]:
df_aux = df[~df.Número.str.contains('Totales')]
df_aux

,Número,Reng,Emisión,Cliente,Vendedor,Almacén,Cantidad,Unid.,Precio Unitario,Monto Base,I.V.A.,Otros,Neto,Tipo
0,00001,NaN,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
1,0000006366,1,2022-01-03 09:00:00,304459.0,1.0,1.0,1.0,UNI,8.64,8.64,1.38,0.0,10.02,fa
2,0000006388,1,2022-01-07 11:40:00,18419125.0,6.0,1.0,1.0,UNI,8.91,8.91,1.43,0.0,10.34,fa
4,00005,NaN,"NIPLE PLASTICO 1"" * 13CMS",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
5,0000006369,3,2022-01-03 11:04:00,13306742.0,7.0,1.0,20.0,UNI,115.20,103.68,16.59,0.0,120.27,fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17657,GEN2,CON IVA,GENERICO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
17658,0000003948,1,2021-04-26 11:19:00,13763881,7.0,1.0,1.0,UNI,21.92,21.92,3.51,0.0,25.43,fa
17659,0000004458,15,2021-05-15 11:10:00,15862673,7.0,1.0,1.0,UNI,10.00,10.00,1.60,0.0,11.60,fa
17660,0000005244,2,2021-06-15 10:32:00,15927422,1.0,1.0,1.0,UNI,4.55,4.55,0.00,0.0,4.55,fa


### Eliminación de columnas no relevantes

In [11]:
# Cambiamos el nombre de las columnas
df_aux.columns

Index(['Número', 'Reng', 'Emisión ', 'Cliente ', 'Vendedor', 'Almacén',
       'Cantidad', 'Unid.', 'Precio Unitario ', 'Monto Base', 'I.V.A.',
       'Otros', 'Neto', 'Tipo'],
      dtype='object')

In [12]:
cols = ['num', 'reng', 'fecha', 'cliente', 'vendedor', 'almacen', 'cantidad',
       'und', 'precio', 'base', 'iva', 'otros', 'neto', 'tipo']
df_aux.columns = cols
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33636 entries, 0 to 17661
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   num       33636 non-null  object 
 1   reng      29492 non-null  object 
 2   fecha     33636 non-null  object 
 3   cliente   28925 non-null  object 
 4   vendedor  28925 non-null  float64
 5   almacen   28925 non-null  float64
 6   cantidad  28925 non-null  float64
 7   und       28925 non-null  object 
 8   precio    28925 non-null  float64
 9   base      28925 non-null  float64
 10  iva       28925 non-null  float64
 11  otros     28925 non-null  float64
 12  neto      28925 non-null  float64
 13  tipo      33636 non-null  object 
dtypes: float64(8), object(6)
memory usage: 3.8+ MB


In [13]:
df_aux = df_aux.loc[:,['num', 'fecha', 'cliente', 'vendedor', 'cantidad', 'neto', 'tipo']]
df_aux


,num,fecha,cliente,vendedor,cantidad,neto,tipo
0,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,NaN,NaN,NaN,NaN,fa
1,0000006366,2022-01-03 09:00:00,304459.0,1.0,1.0,10.02,fa
2,0000006388,2022-01-07 11:40:00,18419125.0,6.0,1.0,10.34,fa
4,00005,"NIPLE PLASTICO 1"" * 13CMS",NaN,NaN,NaN,NaN,fa
5,0000006369,2022-01-03 11:04:00,13306742.0,7.0,20.0,120.27,fa
...,...,...,...,...,...,...,...
17657,GEN2,GENERICO,NaN,NaN,NaN,NaN,fa
17658,0000003948,2021-04-26 11:19:00,13763881,7.0,1.0,25.43,fa
17659,0000004458,2021-05-15 11:10:00,15862673,7.0,1.0,11.60,fa
17660,0000005244,2021-06-15 10:32:00,15927422,1.0,1.0,4.55,fa


### Construcción del DF definitivo

Los datos en el DF están agrupados por producto: Una línea tiene la información sobre el producto y las siguientes son las facturas que incluyen el producto. 

El objetivo es obtener un DF con la siguiente información por cada línea:

    - num: Número de factura (columna num)
    - fecha: Fecha de la factura (columna fecha)
    - cliente: Código del cliente (columna cliente)
    - vendedor: Código del vendedor (columna vendedor)
    - cod: Código del producto (columna num cuando el largo <=5 )
    - producto: Descripción del producto (columna fecha)
    - cantidad: Cantidad de producto en la factura (columna cantidad)
    - monto: Monto neto del producto en la factura (columna neto)
    - tipo: Tipo de factura (columna tipo)
    
Para eso, recorremos el DF y lo vamos construyendo.

In [14]:
%%time

result = []

cod = ''
prod = ''

for index, row in df_aux.iterrows(): 
    if len(row['num']) <= 5 :
        cod = row['num']
        prod = row['fecha']
    else:
        dic = {}
        dic['num'] = row['num']
        dic['fecha'] = row['fecha']
        dic['cliente'] = row['cliente']
        dic['vendedor'] = row['vendedor']
        dic['cod'] = cod
        dic['producto'] = prod
        dic['cantidad'] = row['cantidad']
        dic['monto'] = row['neto']   
        dic['tipo'] = row['tipo'] 
        
        result.append(dic)


CPU times: user 2.66 s, sys: 3.85 ms, total: 2.66 s
Wall time: 2.67 s


In [15]:
df_ventas = pd.DataFrame(result)
df_ventas.sample(30)

,num,fecha,cliente,vendedor,cod,producto,cantidad,monto,tipo
28815,0000003151,2021-03-29 14:22:00,19778553,13.0,06318,BOTAS #39#40#41#42#43 SEGURIDAD ROBUSTA,1.0,65.00,fa
11271,0000003489,2021-10-11 15:51:00,9333703.0,3.0,05001,"POLEA ALUMINIO 2CANAL 3"" TIPO B CORREA 5-8",1.0,31.41,ne
18488,0000006277,2021-11-30 15:16:00,310060940,14.0,00523,CUCHILLA DE CORTE PARA DESMALEZADORA 30CM,1.0,20.01,fa
3505,0000002529,2021-09-13 09:08:00,20717405.0,13.0,00528,"TEE REDUCIDA HG 1"" * 3/4""",1.0,11.33,ne
17933,0000005535,2021-06-24 11:36:00,10749197,14.0,00415,"ANILLO HG 1/2""",1.0,1.91,fa
6392,0000002712,2021-09-17 10:45:00,19610823.0,7.0,01244,"REGADOR 3/4"" PLASTICO DIAMETRO 24MTRS",6.0,147.60,ne
10230,0000000183,2021-06-29 15:49:00,20077844.0,11.0,04183,"SOLDADURA 1/8"" HIERRO COLADO WEST ARCO (POR VA...",2.0,16.04,ne
2767,0000005884,2021-12-27 16:49:00,28131571.0,13.0,00395,"RIEGO ABRAZADERA PLASTICA 32MM * 3/4"" AGRO/FP",7.0,41.16,ne
2254,0000003684,2021-10-19 15:00:00,9334369.0,14.0,00312,"RIEGO ABRAZADERA PLASTICA 40MM * 3/4"" AGRO/ F....",4.0,39.32,ne
20382,0000005503,2021-06-23 15:03:00,2813141,11.0,00911,"PUNTO SOLDADURA PEAD DESDE 32MM A 250MM/8""",1.0,16.50,fa


In [16]:
df_ventas.describe()

,vendedor,cantidad,monto
count,28925.000000,28925.000000,28925.000000
mean,9.312083,4.566491,26.880727
std,3.516934,21.901486,113.978329
min,1.000000,0.020000,0.010000
25%,7.000000,1.000000,2.690000
50%,10.000000,1.000000,7.650000
75%,11.000000,3.000000,21.000000
max,14.000000,2000.000000,6133.500000


In [17]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28925 entries, 0 to 28924
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   num       28925 non-null  object        
 1   fecha     28925 non-null  datetime64[ns]
 2   cliente   28925 non-null  object        
 3   vendedor  28925 non-null  float64       
 4   cod       28925 non-null  object        
 5   producto  28925 non-null  object        
 6   cantidad  28925 non-null  float64       
 7   monto     28925 non-null  float64       
 8   tipo      28925 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 2.0+ MB


In [18]:
# Transformamos los tipos de columnas para que se guarden con
# el tipo correcto

df_ventas.num = df_ventas.num.astype(str)
df_ventas.cod = df_ventas.cod.astype(str)
df_ventas.vendedor = df_ventas.vendedor.astype(int)
df_ventas.cliente = df_ventas.cliente.astype(str)

df_ventas.fecha = pd.to_datetime(df_ventas.fecha)

df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28925 entries, 0 to 28924
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   num       28925 non-null  object        
 1   fecha     28925 non-null  datetime64[ns]
 2   cliente   28925 non-null  object        
 3   vendedor  28925 non-null  int64         
 4   cod       28925 non-null  object        
 5   producto  28925 non-null  object        
 6   cantidad  28925 non-null  float64       
 7   monto     28925 non-null  float64       
 8   tipo      28925 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 2.0+ MB


## Tasa de cambio

Para mitigar los efectos de la inflación, los montos, originalmente en Bolívares (Bs), serán transformados a montos en $USD. Para eso, utilizamos un archivo CSV con todas las tasas de cambio diarias. Luego asignamos la tasa correspondiente a cada archivo según la fecha.

El archivo CSV con las tasas de cambio se crea a partir de un [scraper](https://github.com/dchaconoca/proyecto-ventas/blob/master/scraper_tasa_dolar.ipynb)

In [19]:
FILE_DOLAR = 'datos/in/tasa_dolar.csv'

df_dolar = pd.read_csv(f"{BASE_DIR / FILE_DOLAR}", sep=';')
df_dolar

,fecha,alta,baja
0,02-02-2022,4.69,4.70
1,01-02-2022,4.69,4.74
2,31-01-2022,4.73,4.75
3,30-01-2022,4.73,4.74
4,29-01-2022,4.73,4.75
...,...,...,...
806,19-11-2019,25931.00,33034.00
807,18-11-2019,29027.00,30849.00
808,17-11-2019,28277.00,29509.00
809,16-11-2019,28430.00,29229.00


In [20]:
# Convertimos la columna de fecha
# y extraemos solo los datos a partir del 2021

df_dolar['fecha'] = pd.to_datetime(df_dolar['fecha'], yearfirst=False)
df_dolar = df_dolar.query(' fecha > "2020/12/31" ')
df_dolar

,fecha,alta,baja
0,2022-02-02,4.69,4.70
1,2022-01-02,4.69,4.74
2,2022-01-31,4.73,4.75
3,2022-01-30,4.73,4.74
4,2022-01-29,4.73,4.75
...,...,...,...
393,2021-05-01,1178316.62,1303357.48
394,2021-04-01,1051007.98,1201219.02
395,2021-03-01,1036015.05,1088223.91
396,2021-02-01,1042518.04,1099758.03


In [21]:
# Aplicamos la reconversión (eliminar 6 ceros a partir del 01-10 o para montos mayores de 1000)
def conversion(x):
    return (x/1000000 if x>1000 else x)

df_dolar['tasa_dolar'] = df_dolar.apply(lambda row: conversion(row['alta']), axis = 1)
df_dolar

/home/diana/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,fecha,alta,baja,tasa_dolar
0,2022-02-02,4.69,4.70,4.690000
1,2022-01-02,4.69,4.74,4.690000
2,2022-01-31,4.73,4.75,4.730000
3,2022-01-30,4.73,4.74,4.730000
4,2022-01-29,4.73,4.75,4.730000
...,...,...,...,...
393,2021-05-01,1178316.62,1303357.48,1.178317
394,2021-04-01,1051007.98,1201219.02,1.051008
395,2021-03-01,1036015.05,1088223.91,1.036015
396,2021-02-01,1042518.04,1099758.03,1.042518


In [22]:
df_dolar = df_dolar.loc[:, ['fecha', 'tasa_dolar']]
df_dolar

,fecha,tasa_dolar
0,2022-02-02,4.690000
1,2022-01-02,4.690000
2,2022-01-31,4.730000
3,2022-01-30,4.730000
4,2022-01-29,4.730000
...,...,...
393,2021-05-01,1.178317
394,2021-04-01,1.051008
395,2021-03-01,1.036015
396,2021-02-01,1.042518


In [23]:
df_ventas_fin = pd.merge(df_ventas, df_dolar, on='fecha')
df_ventas_fin.sample(30)

,num,fecha,cliente,vendedor,cod,producto,cantidad,monto,tipo,tasa_dolar
9,0000000200,2021-01-09,12491828,11,00166,RIEGO UNION RAPIDA 32MM AGRO,2.0,3.68,fa,3.909239
8,0000000200,2021-01-09,12491828,11,00151,"RIEGO ADAPTADOR MACHO 32MM * 1"" AGRO/MOM",3.0,2.64,fa,3.909239
22,0000001372,2021-02-05,310875820,13,01602,"RAMPLUG PLASTICO NARANJA 2""*3/8""",6.0,0.49,fa,2.713844
12,0000001372,2021-02-05,310875820,13,00132,"TEFLON 3/4""*15M TENUTA",1.0,1.53,fa,2.713844
23,0000001372,2021-02-05,310875820,13,01691,"TORNILLO 8*1-1/2"" AUTOROSCANTE ESTRIA",6.0,0.53,fa,2.713844
39,0000001372,2021-02-05,310875820,13,04808,"ALICATE 6"" CORTE DIAGONAL INSULADO MANGO ROJO",1.0,6.12,fa,2.713844
24,0000001372,2021-02-05,310875820,13,02277,"TORNILLO 6*1"" AUTOROSCANTE ESTRIA",10.0,0.65,fa,2.713844
28,0000001372,2021-02-05,310875820,13,03023,R. BOMBA TOYAMA,1.0,4.03,fa,2.713844
14,0000001372,2021-02-05,310875820,13,00500,"TEIPE ELECTRICO 3/4""*18MTS NEGRO COBRA",1.0,2.30,fa,2.713844
2,0000003927,2021-10-27,4548435.0,14,04055,BOLSA 200 LITROS NEGRA PARA BASURA POR UNIDAD,6.0,5.46,ne,4.430000


In [24]:
# Calculamos el precio en dólares a la tasa del día
def calculo_precio_dolar(monto, tasa):
    return (monto/tasa)

df_ventas_fin['monto_dolar'] = df_ventas_fin.apply(lambda row: calculo_precio_dolar(row['monto'], row['tasa_dolar']), 
                                              axis = 1)

df_ventas_fin.sample(20)

,num,fecha,cliente,vendedor,cod,producto,cantidad,monto,tipo,tasa_dolar,monto_dolar
10,0000000205,2021-01-09,28980336,1,00271,"PVC SEMI CODO 75MM/3"" AGUAS NEGRAS AMARILLO",1.0,1.67,fa,3.909239,0.427193
34,0000001372,2021-02-05,310875820,13,04407,0,1.0,2.91,fa,2.713844,1.072280
32,0000001372,2021-02-05,310875820,13,04088,PROBADOR DE CORRIENTE STANLEY,1.0,5.36,fa,2.713844,1.975058
29,0000001372,2021-02-05,310875820,13,03191,LLAVE COMBINADA 7MM BRUFER/PROMI,1.0,2.30,fa,2.713844,0.847506
17,0000001372,2021-02-05,310875820,13,00879,LLAVE COMBINADA 17MM ACESA/PROMI,1.0,14.11,fa,2.713844,5.199266
20,0000001372,2021-02-05,310875820,13,01184,CINTA METALICA ACERO INOXIDABLE POR METRO,1.0,2.72,fa,2.713844,1.002268
41,0000001372,2021-02-05,310875820,13,05123,0,1.0,3.83,fa,2.713844,1.411282
1,0000003927,2021-10-27,4548435.0,14,03653,CONECTOR PARA CABLE COAXIAL ROJO 1*100 RG6,6.0,30.90,ne,4.430000,6.975169
6,0000000002,2021-01-26,2809020.0,11,02307,"LLAVE 4""/110MM COMPUERTA BRIDADA HF 150PSI",1.0,475.00,ne,1.740756,272.869953
37,0000001372,2021-02-05,310875820,13,04599,0,1.0,7.87,fa,2.713844,2.899945


## Salvaguarda del DF

In [25]:
%%time
# Guardamos el DF limpio para su análisis posterior

df_ventas_fin.to_parquet(f"{BASE_DIR / 'datos/out/ventas.parquet'}", 
                    compression='GZIP',
                    engine='pyarrow')

df_ventas_fin.to_csv(f"{BASE_DIR / 'datos/out/ventas.csv'}", sep=';')


# Guardamos el DF en un archivo Excel para utilizar los datos en Google Data Studio
df_ventas_fin.to_excel(f"{BASE_DIR / 'datos/out/ventas.xlsx'}")


CPU times: user 45.2 ms, sys: 3.87 ms, total: 49.1 ms
Wall time: 53.3 ms
